In [1]:
import time
import os
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import scipy as sc
import numpy as np
import requests

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import chromedriver_binary  # Adds chromedriver binary to path

In [ ]:
! pip install selenium

In [ ]:
pip install chromedriver-binary

In [ ]:
pip install webdriver-manager

In [2]:
def load_soup_object(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content,"html.parser")
    #soup = BeautifulSoup(open(html_file_name),"html.parser")
    return soup

In [3]:
#def initialize_arraies():
aniNames=[]
formats=[]
number_of_episodes=[]
episode_duration=[]
status=[]
studios=[]
favorites=[]
popularity=[]
genres=[]
mean_score=[]
source=[]
creator=[]

completed=[]
current=[]
planning=[]
paused=[]
dropped=[]

arrayOfArrays = ([aniNames, formats, number_of_episodes, episode_duration,
                  status, studios, favorites,popularity,genres,mean_score,
                  source,creator,completed,current,planning,paused,dropped])

In [4]:
def scrap(url):
    #ani_soup = load_soup_object('https://anilist.co/search/anime/popular')
    ani_soup = BeautifulSoup(url)

    count = 0
    mtag = ani_soup.find_all("div",attrs={"class":"media-card"})
    for t in mtag:
        count+=1
        anime_name = t.find('a',attrs={"class":"title"}).get_text().replace('\n','').replace('\t','')
        aniNames.append(anime_name)

        ani_url = 'https://anilist.co' + t.find('a',attrs={"class":"title"}).get('href')
        soup = load_soup_object(ani_url)
        anime_tag = soup.find_all("div",attrs={"class":"data-set"})
        for x in anime_tag:
            try:
                div_type = x.find("div",attrs={"class":"type"}).get_text()
                div_value = x.find("div",attrs={"class":"value"}).get_text()
            except:
                continue

            if(div_type == 'Format'):
                formats.append(div_value.replace('\n',''))

            if(div_type == 'Episodes'):
                number_of_episodes.append(div_value)

            if('Duration' in div_type):
                episode_duration.append(div_value)
                
            if(div_type == 'Status'):
                status.append(div_value)    
                
            if(div_type == 'Mean Score'):
                mean_score.append(div_value.replace('%',''))

            if(div_type == 'Popularity'):
                popularity.append(div_value)
                
            if(div_type == 'Favorites'):
                favorites.append(div_value)

            if(div_type == 'Studios'):
                studios.append(x.find("a").get_text().replace("\n",""))

            if(div_type == 'Source'):
                source.append(div_value)

            if(div_type == 'Genres'):
                generes_string = ''
                for g in x.find_all("span"):
                    generes_string += g.get_text() + '|'
                generes_string = generes_string[:-1]
                genres.append(generes_string)


        status_tag = soup.find_all("div",attrs={"class":"status"})
        for l in status_tag:
            status_name = l.find("div",attrs={"class":"name"}).get_text()
            status_value = l.find("div",attrs={"class":"amount"}).get_text().replace("Users","").replace("\n","")

            if('Completed' in status_name):
                completed.append(status_value)

            if('Current' in status_name):
                current.append(status_value)

            if('Planning' in status_name):
                planning.append(status_value)

            if('Paused' in status_name):
                paused.append(status_value)

            if('Dropped' in status_name):
                dropped.append(status_value)
                
        roles = soup.find_all('div',attrs={"class":"view-staff"}) #role-card view-staff small
        try:
            creator.append(roles[0].find('div',attrs={"class":"name"}).get_text().replace('\n',''))
        except:
            pass
         
        #after every anime check if there is any missing value and if it is missing, insert NaN
        for i in range(17):
            if((len(arrayOfArrays[i])<count)):
                arrayOfArrays[i].append(np.nan)
                
        time.sleep(1)

In [ ]:
#https://stackoverflow.com/questions/53701759/scroll-with-keys-page-down-in-selenium-python
driver=webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://anilist.co/search/anime/popular")
# The higher time we define, the more Data we will gain through the Crawling.
timeout = time.time() + 60*20  
while True:
    driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
    time.sleep(3)
    if time.time() > timeout:
        #initialize_arrays()
        scrap(driver.page_source)
        break

data = {'anime_name': aniNames, 'Format':formats, 'Number Of Episodes':number_of_episodes,
        "Episode Duration":episode_duration, "status":status, "Popularity":popularity,
        "favorites":favorites,"Studios":studios, "Genres":genres, "mean_score":mean_score,
        "source":source,"Creator":creator,'Completed':completed,'Current':current,
        'Planning':planning,'Paused':paused,'Dropped':dropped}

df = pd.DataFrame(data)
driver.quit()  # Close the browser

In [6]:
df

,anime_name,Format,Number Of Episodes,Episode Duration,status,Popularity,favorites,Studios,Genres,mean_score,source,Creator,Completed,Current,Planning,Paused,Dropped
0,Shingeki no Kyojin,TV,25.0,24 mins,Finished,522270.0,47674.0,Wit Studio,Action|Drama|Fantasy|Mystery,85%,Manga,Hajime Isayama,436995.0,30979.0,36842.0,9191.0,8263.0
1,DEATH NOTE,TV,37.0,23 mins,Finished,474846.0,35688.0,MADHOUSE,Mystery|Psychological|Supernatural|Thriller,84%,Manga,Tsugumi Ooba,369357.0,26960.0,47004.0,16808.0,14717.0
2,Boku no Hero Academia,TV,13.0,24 mins,Finished,471660.0,23789.0,bones,Action|Adventure|Comedy,79%,Manga,Kouhei Horikoshi,388606.0,29172.0,39132.0,7484.0,7266.0
3,Kimetsu no Yaiba,TV,26.0,24 mins,Finished,468624.0,40432.0,ufotable,Action|Adventure|Drama|Fantasy|Supernatural,85%,Manga,Koyoharu Gotouge,362290.0,41180.0,48672.0,9883.0,6599.0
4,HUNTER×HUNTER (2011),TV,148.0,24 mins,Finished,433766.0,56845.0,MADHOUSE,Action|Adventure|Fantasy,90%,Manga,Yoshihiro Togashi,258701.0,65347.0,72050.0,28800.0,8868.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6075,NOMAD: Megalo Box 2 - Hachidori to tabibito.,Special,1.0,NaN,Finished,1225.0,11.0,NaN,NaN,72%,Original,NaN,81.0,73.0,1046.0,16.0,9.0
6076,The Big O (2003),TV,13.0,24 mins,Finished,1225.0,54.0,Sunrise,Action|Mecha|Mystery|Psychological|Sci-Fi,73%,Original,Hajime Yatate,818.0,55.0,326.0,17.0,9.0
6077,Zoids Wild,TV,50.0,25 mins,Finished,1225.0,10.0,OLM,Action|Adventure|Mecha|Sci-Fi,51%,Original,Norihiko Sudo,157.0,135.0,621.0,87.0,225.0
6078,Sanctuary,OVA,1.0,"1 hour, 7 mins",Finished,1225.0,9.0,Pastel,Action|Drama|Thriller,58%,Manga,Yoshiyuki Okamura,503.0,18.0,672.0,16.0,16.0
